In [2]:
import pandas as pd

In [3]:
df_train = pd.read_csv(r"train.csv")

/var/folders/j3/n42l5tvd41v6tn5x11gsbmbw0000gn/T/ipykernel_78552/2563883028.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(r"train.csv")


In [4]:
df_test = pd.read_csv(r"test.csv")

/var/folders/j3/n42l5tvd41v6tn5x11gsbmbw0000gn/T/ipykernel_78552/3082698597.py:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv(r"test.csv")


In [5]:
ID = df_test['ID']

In [6]:
# df_train.info()

In [7]:
# valeurs_manquantes = df_train[['Fuel consumption ', 'Ft']].isnull().sum()
# print("Valeurs manquantes :\n", valeurs_manquantes)

# valeurs_negatives = df_train[df_train['Fuel consumption '] < 0]
# print("\nLignes où 'Fuel consumption' est négatif :\n", valeurs_negatives)

# valeurs_uniques_ft = df_train['Ft'].nunique()
# print("Nombre de valeurs uniques dans la colonne 'Ft' :", valeurs_uniques_ft)

# modalites_ft = df_train['Ft'].unique()
# print("Différentes modalités de la colonne 'Ft' :", modalites_ft)

In [8]:
df_train.loc[(df_train['Ft'] == 'ELECTRIC') & df_train['Fuel consumption '].isnull(), 'Fuel consumption '] = 0
df_test.loc[(df_test['Ft'] == 'ELECTRIC') & df_test['Fuel consumption '].isnull(), 'Fuel consumption '] = 0

# print(df_train[['Ft', 'Fuel consumption ']])

In [9]:
# valeurs_manquantes = df_train['Fuel consumption '].isnull().sum()
# print("Valeurs manquantes :\n", valeurs_manquantes)

In [10]:
# modalites_ft = ['PETROL', 'PETROL/ELECTRIC', 'DIESEL', 'LPG', 'ELECTRIC', 'NG', 'E85',
#                 'DIESEL/ELECTRIC', 'NG-BIOMETHANE', 'HYDROGEN', 'UNKNOWN']

# for modalite in modalites_ft:
#     subset_df = df_train[df_train['Ft'] == modalite]
#     description = subset_df['Electric range (km)'].describe()
#     print(f"\nDistribution pour 'Ft' = {modalite} :\n{description}")

In [11]:
df_train.loc[df_train['Ft'].isin(['PETROL', 'DIESEL','E85', 'HYDROGEN', 'LPG', 'NG', 'NG-BIOMETHANE']), 'Electric range (km)'] = 0

types_carburant = df_train['Ft'].unique()

for carburant in types_carburant:
    median_value = df_train[df_train['Ft'] == carburant]['Electric range (km)'].median()
    df_train.loc[(df_train['Ft'] == carburant) & df_train['Electric range (km)'].isnull(), 'Electric range (km)'] = median_value

# print(df_train[['Ft', 'Electric range (km)']])

In [12]:
df_test.loc[df_test['Ft'].isin(['PETROL', 'DIESEL','E85', 'HYDROGEN', 'LPG', 'NG', 'NG-BIOMETHANE']), 'Electric range (km)'] = 0

types_carburant = df_test['Ft'].unique()

for carburant in types_carburant:
    median_value = df_test[df_test['Ft'] == carburant]['Electric range (km)'].median()
    df_test.loc[(df_test['Ft'] == carburant) & df_test['Electric range (km)'].isnull(), 'Electric range (km)'] = median_value

# print(df_test[['Ft', 'Electric range (km)']])

In [13]:
# types_carburant_a_imputer_0 = ['E85', 'HYDROGEN', 'LPG', 'NG', 'NG-BIOMETHANE']

# df_train.loc[df_train['Ft'].isin(types_carburant_a_imputer_0), 'Electric range (km)'] = 0

# print(df_train[['Ft', 'Electric range (km)']])

In [14]:
# nb_na_electric_range = df_train['Electric range (km)'].isnull().sum()

# print("Nombre de NA dans 'Electric range (km)' :", nb_na_electric_range)

In [15]:
# pourcentage_zeros = (df_train['Electric range (km)'] == 0).mean() * 100

# print("Pourcentage de zéros dans 'Electric range (km)' :", pourcentage_zeros)

In [16]:
# correlation = df_train[['Electric range (km)', 'Ewltp (g/km)']].corr().iloc[0, 1]

# print("Corrélation entre 'Electric range (km)' et 'Ewltp (g/km)' :", correlation)

In [17]:
# df_train['EstElectrique'] = (df_train['Electric range (km)'] > 0).astype(int)
# df_test['EstElectrique'] = (df_test['Electric range (km)'] > 0).astype(int)

# print(df_train[['Electric range (km)', 'EstElectrique']])

In [18]:
condition_est_electrique_hybride_non_electrique = (
    (df_train['Electric range (km)'] > 0) | 
    (df_train['Ft'].str.contains('/ELECTRIC|HYBRID', case=False, na=False)) |
    (df_train['Ft'].str.contains('ELECTRIC', case=False, na=False))
)

condition_hybride = df_train['Ft'].str.contains('DIESEL/ELECTRIC|PETROL/ELECTRIC', case=False, na=False)
df_train.loc[condition_hybride, 'EstType'] = 1

df_train['EstType'].fillna(0, inplace=True)  # Remplir les NaN avec 0 (Non électrique)

# pourcentage_modalites = df_train['EstType'].value_counts(normalize=True) * 100

# print(pourcentage_modalites)

In [19]:
condition_est_electrique_hybride_non_electrique = (
    (df_test['Electric range (km)'] > 0) | 
    (df_test['Ft'].str.contains('/ELECTRIC|HYBRID', case=False, na=False)) |
    (df_test['Ft'].str.contains('ELECTRIC', case=False, na=False))
)

condition_hybride = df_test['Ft'].str.contains('DIESEL/ELECTRIC|PETROL/ELECTRIC', case=False, na=False)
df_test.loc[condition_hybride, 'EstType'] = 1

df_test['EstType'].fillna(0, inplace=True)  # Remplir les NaN avec 0 (Non électrique)

# pourcentage_modalites = df_test['EstType'].value_counts(normalize=True) * 100

# print(pourcentage_modalites)

In [19]:
# correlation = df_train[['EstType', 'Ewltp (g/km)']].corr().iloc[0, 1]

# print("Corrélation entre 'EstType' et 'Ewltp (g/km)' :", correlation)

In [20]:
# df_train.info()

In [21]:
# correlation = df_train[['z (Wh/km)', 'Ewltp (g/km)']].corr().iloc[0, 1]

# print("Corrélation entre 'z (Wh/km)' et 'Ewltp (g/km)' :", correlation)

In [22]:
# valeurs_nulles_electric_range = df_train[df_train['z (Wh/km)'].isnull()]

# modalites_ft_electric_range_nulles = valeurs_nulles_electric_range['Ft'].unique()
# print("Modalités de 'Ft' lorsque 'z (Wh/km)' est manquante :", modalites_ft_electric_range_nulles)

In [20]:
df_train.loc[df_train['Ft'].isin(['PETROL', 'DIESEL','E85', 'HYDROGEN', 'LPG', 'NG', 'NG-BIOMETHANE']), 'z (Wh/km)'] = 0

types_carburant = df_train['Ft'].unique()

for carburant in types_carburant:
    median_value = df_train[df_train['Ft'] == carburant]['z (Wh/km)'].median()
    df_train.loc[(df_train['Ft'] == carburant) & df_train['z (Wh/km)'].isnull(), 'z (Wh/km)'] = median_value

# print(df_train[['Ft', 'z (Wh/km)']])

In [21]:
df_test.loc[df_train['Ft'].isin(['PETROL', 'DIESEL','E85', 'HYDROGEN', 'LPG', 'NG', 'NG-BIOMETHANE']), 'z (Wh/km)'] = 0

types_carburant = df_test['Ft'].unique()

for carburant in types_carburant:
    median_value = df_test[df_train['Ft'] == carburant]['z (Wh/km)'].median()
    df_test.loc[(df_test['Ft'] == carburant) & df_test['z (Wh/km)'].isnull(), 'z (Wh/km)'] = median_value

# print(df_train[['Ft', 'z (Wh/km)']])

/var/folders/j3/n42l5tvd41v6tn5x11gsbmbw0000gn/T/ipykernel_78552/441265878.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  median_value = df_test[df_train['Ft'] == carburant]['z (Wh/km)'].median()


In [25]:
# valeurs_nulles_electric_range = df_train[df_train['z (Wh/km)'].isnull()]

# modalites_ft_electric_range_nulles = valeurs_nulles_electric_range['Ft'].unique()
# print("Modalités de 'Ft' lorsque 'z (Wh/km)' est manquante :", modalites_ft_electric_range_nulles)

In [26]:
# types_carburant_a_imputer_0 = ['E85', 'HYDROGEN', 'LPG', 'NG', 'NG-BIOMETHANE']

# df_train.loc[df_train['Ft'].isin(types_carburant_a_imputer_0), 'z (Wh/km)'] = 0

# # Vérifier le résultat
# print(df_train[['Ft', 'z (Wh/km)']])

In [27]:
# correlation = df_train[['z (Wh/km)', 'Ewltp (g/km)']].corr().iloc[0, 1]

# print("Corrélation entre 'z (Wh/km)' et 'Ewltp (g/km)' :", correlation)

In [28]:
# nb_modalites_IT = df_train['IT'].nunique()

# print("Nombre de modalités uniques dans la colonne 'IT' :", nb_modalites_IT)

In [29]:
# na_both_columns = df_train[df_train['IT'].isnull() & df_train['Erwltp (g/km)'].isnull()]

# pourcentage_na_both_columns = (len(na_both_columns) / len(df_train)) * 100

# print(f"Pourcentage de lignes où à la fois 'IT' et 'Erwltp (g/km)' sont NaN : {pourcentage_na_both_columns:.2f}%")

In [22]:
median_values = df_train.groupby('IT')['Erwltp (g/km)'].median().to_dict()
df_train['Erwltp (g/km)'] = df_train.apply(lambda row: median_values.get(row['IT']) if pd.isna(row['Erwltp (g/km)']) else row['Erwltp (g/km)'], axis=1)

# print(df_train[['IT', 'Erwltp (g/km)']])

In [23]:
median_values = df_test.groupby('IT')['Erwltp (g/km)'].median().to_dict()
df_test['Erwltp (g/km)'] = df_test.apply(lambda row: median_values.get(row['IT']) if pd.isna(row['Erwltp (g/km)']) else row['Erwltp (g/km)'], axis=1)

# print(df_train[['IT', 'Erwltp (g/km)']])

In [35]:
# df_train.info()

In [24]:
df_train['Erwltp (g/km)'] = df_train['Erwltp (g/km)'].fillna('NA')

df_train['Erwltp (g/km)'] = pd.Categorical(df_train['Erwltp (g/km)'])
df_train['Erwltp (g/km)'] = df_train['Erwltp (g/km)'].astype('object')


In [25]:
df_test['Erwltp (g/km)'] = df_test['Erwltp (g/km)'].fillna('NA')

df_test['Erwltp (g/km)'] = pd.Categorical(df_test['Erwltp (g/km)'])
df_test['Erwltp (g/km)'] = df_test['Erwltp (g/km)'].astype('object')


In [26]:
na_percentage = (df_train.isna().mean() * 100).round(2)

columns_with_high_na = na_percentage[na_percentage >= 36]
print(columns_with_high_na)


MMS              100.00
Enedc (g/km)      83.84
IT                37.78
Ernedc (g/km)    100.00
De               100.00
Vf               100.00
dtype: float64


In [27]:
na_percentage = (df_test.isna().mean() * 100).round(2)

columns_with_high_na = na_percentage[na_percentage >= 33]
print(columns_with_high_na)
# df_train = df_train.drop(columns=columns_with_high_na.index)

MMS              100.00
Enedc (g/km)      83.88
IT                37.69
Ernedc (g/km)    100.00
De               100.00
Vf               100.00
dtype: float64


In [28]:
na_percentage = (df_train.isna().mean() * 100).round(2)

columns_with_high_na = na_percentage[na_percentage >= 36]
df_train = df_train.drop(columns=columns_with_high_na.index)

In [29]:
na_percentage = (df_test.isna().mean() * 100).round(2)

columns_with_high_na = na_percentage[na_percentage >= 36]

df_test = df_test.drop(columns=columns_with_high_na.index)

In [30]:
fillna_categorical_columns = ['VFN', 'Mp', 'Tan', 'T', 'Va', 'Ve', 'Mk', 'Cn', 'Ct', 'Fm']

for column in fillna_categorical_columns:
    mode_value = df_train[column].mode()[0]
    df_train[column].fillna(mode_value, inplace=True)

In [31]:
fillna_categorical_columns = ['VFN', 'Mp', 'Tan', 'T', 'Va', 'Ve', 'Mk', 'Cn', 'Ct', 'Ft','Fm']
for column in fillna_categorical_columns:
    mode_value = df_test[column].mode()[0]
    df_test[column].fillna(mode_value, inplace=True)

In [32]:
columns_to_drop = ['Date of registration','r','Status','ID']
df_train = df_train.drop(columns=columns_to_drop)

In [33]:
columns_to_drop = ['Date of registration','r','Status','ID']
df_test = df_test.drop(columns=columns_to_drop)

In [34]:
df_merged = pd.concat([df_train, df_test], ignore_index=True)

In [35]:
numeric_columns_with_missing_values = df_train.select_dtypes(include=['float64']).columns[df_train.select_dtypes(include=['float64']).isna().any()].tolist()

median_values = df_merged[numeric_columns_with_missing_values].median()
df_train[numeric_columns_with_missing_values] = df_train[numeric_columns_with_missing_values].fillna(median_values)


In [36]:
numeric_columns_with_missing_values = df_test.select_dtypes(include=['float64']).columns[df_test.select_dtypes(include=['float64']).isna().any()].tolist()


median_values = df_merged[numeric_columns_with_missing_values].median()
df_test[numeric_columns_with_missing_values] = df_test[numeric_columns_with_missing_values].fillna(median_values)

In [37]:
target = df_train['Ewltp (g/km)']

In [38]:
categorical_columns = ['VFN', 'Mp', 'Mh', 'Man', 'Tan', 'T', 'Va', 'Ve', 'Mk', 'Cn', 'Ct', 'Cr', 'Ft', 'Fm']
selected_categorical_columns_train = [col for col in categorical_columns if df_train[col].nunique() < 30]

df_one_hot_train = pd.get_dummies(df_train[selected_categorical_columns_train], drop_first=True)

In [39]:
selected_categorical_columns_test = [col for col in categorical_columns if df_test[col].nunique() < 30]

df_one_hot_test = pd.get_dummies(df_test[selected_categorical_columns_test], drop_first=True)

In [40]:
df_train = pd.concat([df_train, df_one_hot_train], axis=1)
df_test = pd.concat([df_test, df_one_hot_test], axis=1)

In [41]:
missing_columns_test = set(df_one_hot_train.columns) - set(df_one_hot_test.columns)
missing_columns_test = [col for col in missing_columns_test if col in df_test.columns]

df_test.drop(columns=missing_columns_test, inplace=True)

In [42]:
columns_to_drop_train = set(df_train.columns) - set(df_test.columns)
df_train.drop(columns=columns_to_drop_train, inplace=True)

In [43]:
columns_to_drop_test = set(df_test.columns) - set(df_train.columns)
df_test.drop(columns=columns_to_drop_test, inplace=True)

In [44]:
df_train['Ewltp (g/km)'] = target

In [45]:
from category_encoders import CountEncoder

categorical_columns_with_values = df_train.select_dtypes(include=['object']).columns[df_train.select_dtypes(include=['object']).nunique() > 1].tolist()

count_encoder = CountEncoder(cols=categorical_columns_with_values)

df_encoded_train = count_encoder.fit_transform(df_train.drop('Ewltp (g/km)', axis=1))

In [46]:
df_encoded_test = count_encoder.transform(df_test)

In [40]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

X_train, X_val, y_train, y_val = train_test_split(df_encoded_train, target, test_size=0.2, random_state=42)

xgb_model = xgb.XGBRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)

xgb_model.fit(X_train, y_train)

y_val_pred = xgb_model.predict(X_val)
mae_val = mean_absolute_error(y_val, y_val_pred)
print('Mean Absolute Error on validation set:', mae_val)

Mean Absolute Error on validation set: 3.430211630489446


In [49]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

X_train, X_val, y_train, y_val = train_test_split(df_encoded_train, target, test_size=0.2, random_state=42)

columns_to_remove = ['Ft_NG-BIOMETHANE', 'Ct_N1', 'Ft_DIESEL/ELECTRIC', 'Ft_E85', 'Ft_HYDROGEN']

X_train_filtered = X_train.drop(columns=columns_to_remove, errors='ignore')
X_val_filtered = X_val.drop(columns=columns_to_remove, errors='ignore')

xgb_model = xgb.XGBRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)

xgb_model.fit(X_train_filtered, y_train)

y_val_pred = xgb_model.predict(X_val_filtered)
mae_val = mean_absolute_error(y_val, y_val_pred)

print('Mean Absolute Error on validation set:', mae_val)

Mean Absolute Error on validation set: 3.4261220686296032
